In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import transbigdata as tdb
def station2staion_no_line(station):
    station_no_line = pd.DataFrame()
    for i in range(len(station['站点名'].unique())):
        spec_time_place_station = station[station['站点名'] == station['站点名'].unique()[i]]
        # spec_time_place_station = spec_time_place_station[['开始时间','线路名','站点名','进站量','出站量']]
        spec_time_place_station = spec_time_place_station.fillna(0)
        spec_time_place_station = spec_time_place_station.groupby(['日期','开始时间','结束时间']).agg({
            '站点名': 'first',  # 假设每个时间段内站点名是相同的，取第一个即可
            '进站量': 'sum',
            '出站量': 'sum'
        })
        spec_time_place_station = spec_time_place_station.reset_index()
        station_no_line = pd.concat([station_no_line, spec_time_place_station]).reset_index(drop=True)
    return station_no_line

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
line,stop = tbd.getbusdata('北京',['1号线', '2号线', '5号线', '6号线', '7号线', '8号线', '9&房山线','房山线', '10号线',
       '13号线', '15号线', '昌平线', '首都机场线'])
line = line[line['line']!='大兴机场大巴首都机场线'].reset_index(drop=True)
stop = stop[stop['line']!='大兴机场大巴首都机场线'].reset_index(drop=True)
line = line[line['line']!='首都机场大巴昌平线'].reset_index(drop=True)
stop = stop[stop['line']!='首都机场大巴昌平线'].reset_index(drop=True)
stop.loc[stop['stationnames'] == '清河站', 'stationnames'] = '清河'

Obtaining city id: 北京success
Get bus data: 1号线
地铁1号线八通线(环球度假区-古城) success
地铁1号线八通线(古城-环球度假区) success
地铁1号线支线(八角游乐园-青龙湖东) success
地铁1号线支线(青龙湖东-八角游乐园) success
Get bus data: 2号线
地铁2号线外环(西直门-西直门) success
地铁2号线内环(积水潭-积水潭) success
地铁6号线二期(潞城-东小营南) success
地铁6号线二期(东小营南-潞城) success
Get bus data: 5号线
地铁5号线(宋家庄-天通苑北) success
地铁5号线(天通苑北-宋家庄) success
Get bus data: 6号线
地铁6号线(金安桥-潞城) success
地铁6号线(潞城-金安桥) success
Get bus data: 7号线
地铁7号线(北京西站-环球度假区) success
地铁7号线(环球度假区-北京西站) success
Get bus data: 8号线
地铁8号线(朱辛庄-瀛海) success
地铁8号线(瀛海-朱辛庄) success
Get bus data: 9&房山线
地铁9号线(国家图书馆-郭公庄) success
地铁9号线(郭公庄-国家图书馆) success
Get bus data: 房山线
地铁房山线(东管头南-阎村东) success
地铁房山线(阎村东-东管头南) success
大兴机场大巴房山线(房山区交通局-大兴机场) success
大兴机场大巴房山线(大兴机场-房山区交通局) success
Get bus data: 10号线
地铁10号线外环(车道沟-车道沟) success
地铁10号线内环(巴沟-巴沟) success
Get bus data: 13号线
地铁13号线(东直门-西直门) success
地铁13号线(西直门-东直门) success
地铁13A号线(天通苑东-车公庄) success
地铁13A号线(车公庄-天通苑东) success
Get bus data: 15号线
地铁15号线(清华东路西口-俸伯) success
地铁15号线(俸伯-清华东路西口) success
Get bus 

In [4]:
import pandas as pd
import networkx as nx

line_data = {'线路名': [], '站点名': [], '顺序': []}
for i in range(len(stop)):
    线路名 = line_data['线路名']
    线路名.append(stop.loc[i, 'line'])
    站点名 = line_data['站点名']
    站点名.append(stop.loc[i, 'stationnames'])
    顺序 = line_data['顺序']
    顺序.append(stop.loc[i, 'id'])
    line_data.update({'线路名':线路名, '站点名':站点名, '顺序':顺序})
# 示例数据
# line_data = {
#     '线路名': ['1号线', '1号线', '1号线', '2号线', '2号线'],
#     '站点名': ['站点A', '站点B', '站点C', '站点B', '站点D'],
#     '顺序': [1, 2, 3, 1, 2]
# }


stop_data = {'站点名':list(stop['stationnames'].unique())}
# 示例数据
# stop_data = {
#     '站点名': ['站点A', '站点B', '站点C', '站点D'],
#     '其他信息': ['infoA', 'infoB', 'infoC', 'infoD']
# }

line_for_adj = pd.DataFrame(line_data)
stop_for_adj = pd.DataFrame(stop_data)

# 构建地铁网络图
G = nx.Graph()

# 添加节点
for station_for_adj in stop_for_adj['站点名']:
    G.add_node(station_for_adj)

# 添加边（根据站点顺序）
for line_name, group in line_for_adj.groupby('线路名'):
    sorted_stations = group.sort_values('顺序')['站点名'].tolist()
    for i in range(len(sorted_stations) - 1):
        G.add_edge(sorted_stations[i], sorted_stations[i + 1])

# 提取邻接矩阵
adj_matrix = nx.adjacency_matrix(G).todense()

In [17]:
def get_rem_sta_info(origin_df: pd.DataFrame, rem_sta_list: list[str]) -> pd.DataFrame:
    """ 填补单个csv文件中缺失站点的数据 """
    date_str = origin_df.loc[0, '日期'] # 日期与文件中其他所有数据保持一致
    line_str = '' # 线路名不纳入训练，初始化为空字符串
    outflow = type(origin_df.loc[0, '进站量'])(0)
    inflow = type(origin_df.loc[0, '进站量'])(0)
    start_time_enum = list(origin_df['开始时间'].unique())
    end_time_enum = list(origin_df['结束时间'].unique())
    time_point_num = len(start_time_enum)

    output_df = pd.DataFrame(columns=origin_df.columns)
    for time_i in range(time_point_num):
        for rem_sta_i in range(len(rem_sta_list)):
            new_row = pd.DataFrame([{'日期':date_str,
                                     '开始时间':start_time_enum[time_i],
                                     '结束时间':end_time_enum[time_i],
                                     '站点名':rem_sta_list[rem_sta_i],
                                     '线路名':line_str,
                                     '进站量':inflow,
                                     '出站量':outflow
                                     }])
            output_df = pd.concat([output_df, new_row], ignore_index=True)
    return output_df


def get_station_for_adj(stop_for_adj: pd.DataFrame, csv_list: list[str]) -> pd.DataFrame:
    """ 输入邻接矩阵中所有的站点名以及要读入的csv文件名列表， 创建station的dataframe """
    tmp_df = pd.read_csv(csv_list[0])    # 获取列表中首个文件的数据

    # 添加文件数据中缺失的站点数据，目前方案为全部赋值为0
    remain_staion_list = [] # 缺失的站点名列表
    for station_name in list(stop_for_adj['站点名']):
        if not station_name in list(tmp_df['站点名']):
            remain_staion_list.append(station_name)
    print('remain_staion_list is ', len(remain_staion_list))

    output_df_list = []
    for csv_file in csv_list:
        tmp_ori_df = pd.read_csv(csv_file) # 获取原始数据
        output_df_list.append(tmp_ori_df)
        tmp_rem_df = get_rem_sta_info(tmp_ori_df, remain_staion_list) # 填补缺失数据
        output_df_list.append(tmp_rem_df)
        print('tmp_rem_df.__len__ is ', len(tmp_rem_df))

    output_df = pd.concat(output_df_list, axis=0, ignore_index=True)
    print('output_df.__len__ is ', len(output_df))
    return output_df


station = get_station_for_adj(stop_for_adj, ['CTS-2024-dataset/station_20230519.csv'])
station = station2staion_no_line(station)

remain_staion_list is  135
tmp_rem_df.__len__ is  6480
output_df.__len__ is  15264


In [18]:
# station['站点名'].unique().__len__()
print('total station num in G is ', len(stop_for_adj))

total station num in G is  285


In [5]:
from datetime import datetime

sta_info_for_adj = pd.DataFrame(columns=['station_name', 'abs_date', 'time_point', 'day_of_week', 'is_event', 'in_flow', 'out_flow'])
for i in range(len(station)):
    station_name = station.loc[i, '站点名']
    precise_time = datetime.strptime(station.loc[i, '开始时间'], '%Y/%m/%d %H:%M:%S')
    time_diff = precise_time - datetime.strptime('2023/5/19 00:00:00', '%Y/%m/%d %H:%M:%S')
    abs_date = time_diff.days + 1 # 从1开始编号，五月19是第一天
    time_point = (precise_time.hour * 60 + precise_time.minute) / 30
    day_of_week = precise_time.weekday() # Monday == 0 ... Sunday == 6
    event_condition = ((precise_time.day == 26 or precise_time.day == 27) and precise_time.month == 5) or ((precise_time.day == 25) and precise_time.month == 8)
    is_event = 1 if event_condition else 0
    in_flow = float(station.loc[i, '进站量'])
    out_flow = float(station.loc[i, '出站量'])
    sta_info_for_adj.loc[i] = [station_name, abs_date, time_point, day_of_week, is_event, in_flow, out_flow]

In [24]:
categories_list = list(stop_for_adj['站点名'])
my_x = []
my_y = []
for i in sta_info_for_adj.groupby(["abs_date","time_point"]):
    temp = i[1]
    temp['station_name'] = pd.Categorical(temp['station_name'], categories=categories_list, ordered=True)
    temp = temp.sort_values(by=['station_name']).reset_index(drop=True)# 特定日期特定时间的所有站点已知数据
    features = temp[['abs_date','time_point','day_of_week','is_event']].values
    labels = temp[['in_flow','out_flow']].values
    my_x.append(features)
    my_y.append(labels)
# my_x[0]
my_y = np.array(my_y)
my_y.__len__()

48